# 高级合并: 使用pd.merge方法连接数据集

In [1]:
import pandas as pd
import numpy as np

## 1. 数据连接的类型
pd.merge方法实现了三种数据连接的类型：一对一、多对一和多对多。

### 1.1 一对一连接  
与按列合并及其类似

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue','ky'],
                    'hire_date': [2004, 2008, 2012, 2014,2020]})
print(df1, df2, sep='\n\n')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
4       ky       2020


In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


pd.merge()方法会发现两个 DataFrame 都有 "employee" 列，并会自动以这列作为键进行连接。两个输入的合并结果是一个新的 DataFrame。 需要注意的是，共同列的位置可以是不一致的。比如这个例子中，df1和df2中 employee 列的位置是不一样的，pd.merge()方法会正确处理这个问题。

### 1.2 多对一连接

多对一连接是指在需要连接的两个列中，有一个列的值有重复。通过多对一连接获得的结果 DataFrame 将会保留重复值。

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

print(df1, df4, pd.merge(df1,df4),sep='\n\n')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

  employee        group supervisor
0      Bob   Accounting      Carly
1     Jake  Engineering      Guido
2     Lisa  Engineering      Guido
3      Sue           HR      Steve


### 1.3 多对多连接

如果左右两个输入的 DataFrame 包含的共同列都有重复值，这种链接就属于多对多连接。结果保存重复值。

In [11]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
print(df1, df5, pd.merge(df1, df5), sep='\n\n')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## 2. 设置数据合并的键

### 2.1 参数 on 的设置

In [5]:
print(df1, df2, pd.merge(df1, df2, on='employee'), sep='\n\n')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
4       ky       2020

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


### 2.2 left_on 和 right_on  
合并两个列名不同的数据集

In [6]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1, df3, pd.merge(df1, df3, left_on="employee", right_on="name"),sep='\n\n')

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


获取的结果会有一个多余的列，可以通过drop()方法去掉

In [7]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### 2.3. 使用  DataFrame 的**join**方法能够按照索引进行数据合并。

In [9]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [10]:
print(df1a, df2a, df1a.join(df2a),sep='\n\n')

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
ky             2020

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


## 3. 设置数据连接的集合操作规则

In [13]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
print(df6, df7, pd.merge(df6, df7), sep='\n\n')

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

   name   food drink
0  Mary  bread  wine


我们合并两个数据集，在 name 列中只有一个共同的值：Mary。默认情况下，结果中只会包含两个输入集合的交集，这种连接方式被称为**内连接**。我们可以用**how**来设置连接方式，默认为**how = 'inner'**

In [14]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


how参数其他的可选值为：outer、left、right

In [23]:
print(df6, df7, pd.merge(df6, df7, how='outer'),sep='\n\n')

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


**outer**: 外连接，并集

In [16]:
print(df6, df7, pd.merge(df6, df7, how='left'),sep='\n\n')

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


**left**:左连接，以左边的条目为准。

 ## 案例：美国各州的统计数据

**1. 通过pd.read_csv()读取本地数据**

In [17]:
pop = pd.read_csv('data\\data_USstates\\state-population.csv') # 美国各州在1990-2012年间，人口数据
areas = pd.read_csv('data\\data_USstates\\state-areas.csv') # 美国各州的面积
abbrevs = pd.read_csv('data\\data_USstates\\state-abbrevs.csv') # 各州名字及其对应的缩写

In [21]:
# print(pop.head())
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [22]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [23]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [24]:
areas.shape

(52, 2)

In [42]:
pop.shape

(2544, 4)

In [43]:
abbrevs.shape

(51, 2)

**2. 计算美国各州的人口密度并排序**

首先将 abbrevs 和 pop 进行合并，关键字为'state/region'和'abbreviation'，连接方式为 outer（宁可重复，不可缺值）

pd.merge(df1, df2, how="", left_on="", right_on="")  

df1, df2: 进行合并的两个dataframe；  
how：取值为 outer 表示外连接，结果包括两个dataframe 的并集；若是外连接，则结果包括两个dataframe的交集；left 表示左连接，以左边的df为准；right：右连接，以右边的df为准。  

on: 如果df1、df2 中有相同的列，那么可以指定相同列的列名进行合并。如果同一组数据在两个表中的列名不同，就可以通过下面的参数分别指定：  

left_on: df1 中的列名
right_on: df2 中的列名

举例：

In [25]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Secretary', 'HR']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Secretary
3,Sue,HR


In [26]:
df2 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
df2

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [27]:
pd.merge(df1, df2, on="group", how="outer")

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Secretary,NaN
5,Sue,HR,spreadsheets
6,Sue,HR,organization


In [28]:
pd.merge(df1, df2, on="group", how="inner")

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Sue,HR,spreadsheets
5,Sue,HR,organization


In [29]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [30]:
merged = merged.drop(columns=['abbreviation']) # drop duplicate info
merged

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


检查一下数据是否有缺失

In [50]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

我们发现 population 列和 state 列都有缺失值，先来看看population列缺失值的情况。

In [31]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


我们发现所有的人口缺失值都出现在2000年前的PR（波多黎各），应该是统计的时候就没有包含进去。接下来我们看看 state 缺失的情况。

In [33]:
merged.loc[merged['state'].isnull(),'state/region'].unique() #

array(['PR', 'USA'], dtype=object)

我们发现 PR 和 USA 出现了缺失值，也就是说很可能是这两项并没有出现在州名缩写表中。我们应该快速填充对应的全称。

In [36]:
merged.loc[merged['state/region'] == 'PR','state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

现在 state 列没有缺失值了，我们将各州的面积数据也合并进来。

In [37]:
# areas.head()
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Puerto Rico
2449,PR,total,1990,NaN,Puerto Rico
2450,PR,total,1991,NaN,Puerto Rico
2451,PR,under18,1991,NaN,Puerto Rico
2452,PR,total,1993,NaN,Puerto Rico
2453,PR,under18,1993,NaN,Puerto Rico
2454,PR,under18,1992,NaN,Puerto Rico
2455,PR,total,1992,NaN,Puerto Rico
2456,PR,under18,1994,NaN,Puerto Rico
2457,PR,total,1994,NaN,Puerto Rico


In [40]:
final = pd.merge(merged, areas, on='state', how='left') # 以左边表格为主，毕竟如果一个地方只知道面积，没有其他数据没有用
final.head(10)

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
5,AL,total,2011,4801627.0,Alabama,52423.0
6,AL,total,2009,4757938.0,Alabama,52423.0
7,AL,under18,2009,1134192.0,Alabama,52423.0
8,AL,under18,2013,1111481.0,Alabama,52423.0
9,AL,total,2013,4833722.0,Alabama,52423.0


检查一下数据，还有哪些列有缺失值。

In [41]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

除了 population 之外，area 项还有缺失值。

In [46]:
final.loc[final['area (sq. mi)'].isnull(),'state/region'].unique()

array(['USA'], dtype=object)

In [47]:
final.loc[final['population'].isnull(),'state/region'].unique()

array(['PR'], dtype=object)

我们发现缺少了全美国的面积数据，可以插入全国总面积数据（对各州面积求和即可）。但是针对本案例，我们去掉这个缺失值，因为全国人口密度并不需要这个数据。

In [48]:
np.sum(final.loc[final['state/region'] != 'USA','area (sq. mi)'].unique())

3790399.0

In [49]:
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,United States,NaN
2540,USA,under18,2011,73902222.0,United States,NaN
2541,USA,total,2011,311582564.0,United States,NaN
2542,USA,under18,2012,73708179.0,United States,NaN


In [50]:
final.dropna(inplace=True) 
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


现在所有的数据都准备好了。先选择2012年的各州人口以及总人口数据进行计算。

In [51]:
data2012 = final.loc[(final['year'] == 2012) & (final['ages'] == 'total'),['state','population','area (sq. mi)']]
data2012

,state,population,area (sq. mi)
1,Alabama,4817528.0,52423.0
95,Alaska,730307.0,656425.0
97,Arizona,6551149.0,114006.0
191,Arkansas,2949828.0,53182.0
193,California,37999878.0,163707.0
287,Colorado,5189458.0,104100.0
289,Connecticut,3591765.0,5544.0
383,Delaware,917053.0,1954.0
385,District of Columbia,633427.0,68.0
479,Florida,19320749.0,65758.0


接下来我们计算总人口密度并按序排列。

In [52]:
data2012.set_index('state',inplace=True)
data2012

,population,area (sq. mi)
state,,
Alabama,4817528.0,52423.0
Alaska,730307.0,656425.0
Arizona,6551149.0,114006.0
Arkansas,2949828.0,53182.0
California,37999878.0,163707.0
Colorado,5189458.0,104100.0
Connecticut,3591765.0,5544.0
Delaware,917053.0,1954.0
District of Columbia,633427.0,68.0


In [54]:
density = data2012['population'] / data2012['area (sq. mi)']
density

state
Alabama                   91.897221
Alaska                     1.112552
Arizona                   57.463195
Arkansas                  55.466662
California               232.121278
Colorado                  49.850701
Connecticut              647.865260
Delaware                 469.320880
District of Columbia    9315.102941
Florida                  293.815946
Georgia                  166.814926
Hawaii                   127.157885
Idaho                     19.091942
Illinois                 222.179495
Indiana                  179.510763
Iowa                      54.642103
Kansas                    35.067184
Kentucky                 108.379649
Louisiana                 88.770596
Maine                     37.542063
Maryland                 474.318369
Massachusetts            629.588157
Michigan                 102.081593
Minnesota                 61.875551
Mississippi               61.660197
Missouri                  86.423876
Montana                    6.837955
Nebraska              

In [57]:
density.sort_values(ascending=False)

state
District of Columbia    9315.102941
Puerto Rico             1038.846373
New Jersey              1016.710502
Rhode Island             679.808414
Connecticut              647.865260
Massachusetts            629.588157
Maryland                 474.318369
Delaware                 469.320880
New York                 359.359798
Florida                  293.815946
Pennsylvania             277.139151
Ohio                     257.719082
California               232.121278
Illinois                 222.179495
Virginia                 191.414997
North Carolina           181.125657
Indiana                  179.510763
Georgia                  166.814926
Tennessee                153.156029
South Carolina           147.574499
New Hampshire            141.334296
Hawaii                   127.157885
Kentucky                 108.379649
Michigan                 102.081593
Texas                     97.024196
Washington                96.704458
Alabama                   91.897221
Louisiana             

计算结果是美国各州加上华盛顿特区、波多黎各在2012年的人口密度排序，以万人/平方英里为单位。  
人口密度最低的几个州排列如下：

In [58]:
density.tail()

state
Washington         96.704458
West Virginia      76.624159
Wisconsin          87.393768
Wyoming             5.894886
Puerto Rico      1038.846373
dtype: float64